In [ ]:
!pip install transformers
!pip install torch
!pip install accelerate
!pip install flask
!pip install pyngrok

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
# Load the model and tokenizer
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [3]:
from pyngrok import ngrok

# Replace 'YOUR_NGROK_AUTH_TOKEN' with your actual token
ngrok.set_auth_token("you_ngrok_access_token")

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import torch

app = Flask(__name__)

@app.route('/')
def home():
    return "Welcome to the API!"

@app.route('/process', methods=['POST'])
def process_input():
    try:
        # Get user input
        data = request.json
        user_input = data.get('input', '')

        pre_prompt = '''You are an AI designed to generate quizzes. Always respond in the following JSON format:

{
  "quiz_title": "Quiz Title",
  "questions": [
    {
      "question": "Question text here",
      "options": ["Option 1", "Option 2", "Option 3", "Option 4"],
      "correct_answer": "Correct Option"
    }
  ]
}

'''
        text = f'''Generate a 10 question quiz for the following text:
{user_input}

Output:
'''
        prompt = pre_prompt + text
        # Generate output using the model
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        outputs = model.generate(
          **inputs,
          max_length=2024,  # Limit the length of the response
          temperature=0.3, # Controls creativity; lower values = more focused
          top_p=0.3,       # Controls diversity; higher values = more diverse
          do_sample=True   # Enables sampling for varied responses
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Return the result
        data =  response[len(prompt):].strip()

        def extract_and_jsonify(input_string):
            # Find the first opening brace
            start_idx = input_string.find("{")

            if start_idx == -1:
                return "No opening brace found"

            # Find the last closing brace starting from the end of the string
            end_idx = input_string.rfind("}", start_idx)

            if end_idx == -1:
                return "No closing brace found"

            # Extract the content between the braces
            extracted_content = input_string[start_idx:end_idx + 1]

            return extracted_content
            # Try to parse the extracted content into a JSON object
            # try:
            #     # Strip extra spaces and newlines before parsing
            #     json_data = json.loads(extracted_content.strip())
            #     return json_data
            # except json.JSONDecodeError as e:
            #     print(f"Error decoding JSON: {e}")
            #     return "Invalid JSON format"

        jsonified_data = extract_and_jsonify(data)
        return jsonified_data

    except Exception as e:
        # Return an error message as JSON
        return jsonify({"error": str(e)}), 500


if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(f"Public URL: {public_url}")
    app.run(port=5000)


MODEL IS RUNNING WELL BELOW:-

In [ ]:
import json
# input_text = '''Output should be a valid JSON array of objects. Each object should contain a "question", "options" (array of strings), and "correct_answer".
# The response should strictly follow this format without any additional text or explanation. The output should start with '[' and end with ']' and only contain valid objects.
# Generate a series of 10 multiple-choice questions about the topic of Cricket in this JSON format.'''

# input_text = '''I want you to create 10 multiple choice questions based on the topic Cricket. Don't include any explanation or extra text. Follow the following format strictly for each question:
# Question
# Options
# Correct Answer'''
# input_text = '''You are an AI designed to generate quizzes. Always respond in the following JSON format:

# {
#   "quiz_title": "Quiz Title",
#   "questions": [
#     {
#       "question": "Question text here",
#       "options": ["Option 1", "Option 2", "Option 3", "Option 4"],
#       "correct_answer": "Correct Option"
#     }
#   ]
# }

# Generate a 10 question quiz for the following text:
# "Video Games"

# Output:
# '''


pre_prompt = '''You are an AI designed to generate quizzes. Always respond in the following JSON format:

{
  "quiz_title": "Quiz Title",
  "questions": [
    {
      "question": "Question text here",
      "options": ["Option 1", "Option 2", "Option 3", "Option 4"],
      "correct_answer": "Correct Option"
    }
  ]
}

'''
user_input = '''Introduction to Computers
Computers  are  electronic  devices  that  process  data  and  perform  tasks  according  to  a  set  of
instructions, known as programs.
They  are  an  integral  part  of  modern  life,  enabling  advancements  in  science,  business,  healthcare,
education, and entertainment.
The  history  of  computers  dates  back  to  the  19th  century  with  the  invention  of  mechanical  devices
such as Charles Babbage's Analytical Engine.
However,  the  development  of  modern  computers  began  in  the  20th  century  with  the  advent  of
electronic components like vacuum tubes, transistors, and integrated circuits.
A computer system consists of hardware and software. Hardware refers to the physical components,
such as the central processing unit (CPU), memory, storage devices, and input/output devices.
Software  encompasses  the  programs  and  operating  systems  that  run  on  the  hardware,  enabling
users to perform various tasks.
Types  of  computers  include  desktops,  laptops,  servers,  and  supercomputers,  each  designed  for
specific purposes.
Supercomputers,  for  instance,  are  used  for  complex  computations  in  fields  like  climate  modeling,
molecular research, and artificial intelligence.
Programming   languages,   such   as   Python,   Java,   and   C++,   play   a   crucial   role   in   software
development. These languages allow developers to write instructions that computers can execute to
solve problems or perform specific tasks.
In  recent  years,  technologies  like  cloud  computing,  artificial  intelligence,  and  quantum  computing
have  further  revolutionized  the  capabilities  of  computers,  making  them  indispensable  tools  for

innovation and productivity.'''
text = f'''Generate a 10 question quiz for the following text:
"{user_input}"

Output:
'''
input_text = pre_prompt + text

# Tokenize the input and move the tensor to GPU
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Generate output using the model
output = model.generate(inputs["input_ids"],max_length=2024,temperature=0.3,top_p=0.3,do_sample=True)

# Decode the generated output (also on GPU)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
data = output_text[len(input_text):].strip()

def extract_and_jsonify(input_string):
    # Find the first opening brace
    start_idx = input_string.find("{")

    if start_idx == -1:
        return "No opening brace found"

    # Find the last closing brace starting from the end of the string
    end_idx = input_string.rfind("}", start_idx)

    if end_idx == -1:
        return "No closing brace found"

    # Extract the content between the braces
    extracted_content = input_string[start_idx:end_idx + 1]

    # Try to parse the extracted content into a JSON object
    try:
        json_data = json.loads(extracted_content)
        return json_data
    except json.JSONDecodeError:
        return "Invalid JSON format"

jsonified_data = extract_and_jsonify(data)
print(jsonified_data)
